In [67]:
import numpy as np
import skimage 
from skimage import io
from skimage.measure import label, regionprops, regionprops_table
from skimage.segmentation import clear_border, expand_labels
from skimage.color import label2rgb
import scipy
from PIL import Image
import glob
import os
import matplotlib.pyplot as plt

In [73]:
def segmentation(im, num_erosions = 5, diameter_threshold = 300, disk_size = 2):
    '''
    input: raw image
    output: segmented image

    This function segments images of ice floes. We make use of skimage.filters.threshold_otsu, 
    skimage.morphology.binary_erosion, and skimage.morphology.diameter_closing. Erosional steps 
    help to distinguish ice floes from on another. The high diameter threshold fills melt ponds. 
    The disk size is a parameter in the erosion and the connectivity is a parameter in the hole 
    filling. For the helicopter photos from HOTRAX 2005, these defaults worked well, but playing 
    with them might result in more accurate segmentations. 
    '''
    
    # Threshold.
    threshold = skimage.filters.threshold_triangle(im) - 0.075
    simple_threshold = (im < threshold)
    simple_threshold = 1 - simple_threshold

    # Erode.
    i = 0
    eroded_threshold = simple_threshold
    while i < num_erosions:
        eroded_threshold = skimage.morphology.binary_erosion(eroded_threshold, skimage.morphology.disk(disk_size))
        i+=1

    # Fill holes and ponds.
    segmented_im = skimage.morphology.diameter_closing(eroded_threshold, diameter_threshold = diameter_threshold)
    
    # Return segmentation. 
    return segmented_im.astype(int)

later: have preprocess return an image to use in segmentation and just a chopped up color image. try to train model on color image instead

In [74]:
def preprocess(image):
    # Mask
    image_s = np.array(skimage.color.rgb2gray(image))
    image_s = Image.fromarray(image_s)
    MAX_SIZE = (1000, 1000)
    image_s.thumbnail(MAX_SIZE)
    image_s = np.array(image_s)
    image_s = image_s[0:640, 0:896]
    
    # Image
    image.thumbnail(MAX_SIZE)
    image = np.array(image)
    image = image[0:640, 0:896, :]
    return image_s, image

In [76]:
image_dir = '/scratch/groups/earlew/arlenlex/cs230-datasetv2/images/'
out_x_train = '/scratch/groups/earlew/arlenlex/cs230-datasetv2/train/x_trainb/'
out_y_train = '/scratch/groups/earlew/arlenlex/cs230-datasetv2/train/y_trainb/'
out_x_test = '/scratch/groups/earlew/arlenlex/cs230-datasetv2/test/x_testb/'
out_y_test = '/scratch/groups/earlew/arlenlex/cs230-datasetv2/test/y_testb/'

In [77]:
idx = 0
for filename in sorted(glob.glob(os.path.join(image_dir, '*.JPG'))):
    # Segment
    image = Image.open(filename)
    im_s, image = preprocess(image)
    mask = segmentation(im_s)
    name, ext = os.path.splitext(filename)
    name = os.path.basename(name)
    

    # Export
    out_x_train_path = out_x_train + name + '_image'
    out_y_train_path = out_y_train + name + '_mask'
    out_x_test_path = out_x_test + name + '_image'
    out_y_test_path = out_y_test + name + '_mask'
    if idx % 10 != 0:
        np.save(out_x_train_path, image)
        np.save(out_y_train_path, mask)
    else:
        np.save(out_x_test_path, image)
        np.save(out_y_test_path, mask)
    idx+=1
